# 5. 오차역전파법

## 5.4 단순한 계층 구현하기

### 5.4.1 곱셈 계층

In [16]:
class MulLayer:
  def __init__(self):
    self.x = None
    self.y = None

  def forward(self, x, y):
    self.x = x
    self.y = y
    out = x * y
    return out

  def backward(self, dout): #dout : 상류에서 넘어온 미분
    dx = dout * self.y #x와 y를 바꿈
    dy = dout + self.x
    return dx, dy

- 순전파 구현

In [10]:
# 입력값
apple = 100
apple_num = 2
tax = 1.1

# 계층들
mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

#순전파
apple_price = mul_apple_layer.forward(apple, apple_num)
price = mul_tax_layer.forward(apple_price, tax)

print(price)

220.00000000000003


- 역전파 구현 : 각 변수에 대한 미분

In [8]:
dprice = 1
dapple_price, dtax = mul_tax_layer.backward(dprice)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print(dapple, dapple_num, dtax)

2.2 101.1 201


### 5.4.2 덧셈 계층

In [11]:
class AddLayer:
  def __init__(self):
    pass

  def forward(self, x, y):
    out = x + y
    return out

  def backward(self, dout):
    dx = dout * 1
    dy = dout * 1
    return dx, dy

In [17]:
apple = 100
apple_num = 2
orange = 150
orange_num = 3
tax = 1.1

#layer
mul_apple_layer = MulLayer()
mul_orange_layer = MulLayer()
add_total_layer = AddLayer()
mul_tax_layer = MulLayer()

#forward
apple_price = mul_apple_layer.forward(apple, apple_num)
orange_price = mul_orange_layer.forward(orange, orange_num)
total_price = add_total_layer.forward(apple_price, orange_price)
price = mul_tax_layer.forward(total_price, tax)

print(price)

#backward
dprice = 1
dtotal_price, dtax = mul_tax_layer.backward(dprice)
dapple_price, dorange_price = add_total_layer.backward(dtotal_price)
dorange, dorange_num = mul_orange_layer.backward(dorange_price)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print(dapple_num, dapple, dorange, dorange_num, dtax)

715.0000000000001
101.1 2.2 3.3000000000000003 151.1 651


<br>

## 5.5 활성화 함수 계층 구현하기

### 5.5.1 ReLU 계층

In [18]:
class Relu:
  def __init__(self):
    self.mask = None

  def forward(self, x):
    self.mask = (x<=0)
    out = x.copy()
    out[self.mask] = 0
    return out

  def backward(self, dout):
    dout[self.mask] = 0
    dx = dout
    return dx

### 5.5.2 Sigmoid 계층

In [19]:
import numpy as np
class Sigmoid:
  def __init__(self):
    self.out = None

  def forward(self, x):
    out = 1 / (1 + np.exp(-x))
    self.out = out
    return out
  
  def backward(self, dout):
    dx = dout * (1.0 - self.out) * self.out
    return dx

<br>

## 5.6 Affine/Softmax 계층 구현하기

### 5.6.2 배치용 Affine 계층